In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


**We have commented those libraries which you dont need if you skip the augmentation of images part**

We are already giving that data in GNR folder.


In [2]:
import matplotlib.pyplot as plt
import time
import copy
import cv2
import torch
from tqdm.auto import tqdm

# For model building
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
import torch.optim as optim
from torch.optim import lr_scheduler
from torchvision.datasets import ImageFolder
# from torchvision.utils import make_grid
from torch.utils.data import Dataset, DataLoader
import matplotlib.pyplot as plt
import torchvision
from torchvision import datasets, models, transforms
from tqdm import tqdm
import torch
import torch.nn.functional as F
import math
import numpy as np
# import PIL
# from tensorflow import keras
# from tensorflow.keras import layers
# from tensorflow.keras.models import Sequential
import random
# import tensorflow.keras.layers as Layers
# import tensorflow.keras.activations as Actications
# import tensorflow.keras.models as Models
# import tensorflow.keras.optimizers as Optimizer
import os
import shutil
# from tensorflow.keras.optimizers import Adam
# import tensorflow.keras.metrics as Metrics
# import tensorflow.keras.utils as Utils
# from keras.utils.vis_utils import model_to_dot

**Dont run below codes, We already gave you the augmented train data and validation data**

You can save time by skipping over these data augmentation codes

Run skipping 3 cell blocks (containing augmenting code)



**If u decide to run the below augmentating images code, then delete the 'new_train' folder in the given GNR folder because these codes are going to generate it. If u don't, then they will store new generated images in that old old 'new_train' folder and the size of training dataset will double.**

Already it was taking much time in executing it for 3 epoch on training dataset due to system constrainst :(

In [ ]:
# Augmenting all Train folder images
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from tqdm import tqdm
datagen = ImageDataGenerator(
        fill_mode = "constant",
        rotation_range=5,
        shear_range=0.05,
        width_shift_range=0.01,
        zoom_range=0.1,
        height_shift_range=0.01,
        vertical_flip=True,
        horizontal_flip=True)
classes = ['golf_course','crosswalk', 'bridge', 'basketball_court']

In [ ]:
# Generating 12 augmented images from single train image
for cls in classes:
    for filename in tqdm(os.listdir("/content/drive/My Drive/GNR/dataset/train/"+cls)):
        img = load_img("/content/drive/My Drive/GNR/dataset/train/"+cls+"/"+filename)  # this is a PIL image
        x = img_to_array(img)  # this is a Numpy array with shape (3, 150, 150)
        x = x.reshape((1,) + x.shape)  # this is a Numpy array with shape (1, 3, 150, 150)

        i = 0
        for batch in datagen.flow(x, batch_size=1,
                              save_to_dir="/content/drive/My Drive/GNR/dataset/train_new/train/"+cls, save_prefix=filename, save_format='jpeg'):
            i += 1
            if i > 11:
                break  # otherwise the generator would loop indefinitely

In [ ]:
#  Creating Train / Val / Test folders (One time use)

root_dir = "/content/drive/My Drive/GNR/dataset/train"
classes_dir = ['golf_course','crosswalk', 'bridge', 'basketball_court']

val_ratio = 0.20

for cls in classes_dir:
    os.makedirs(root_dir +'/train/' + cls)
    os.makedirs(root_dir +'/val/' + cls)


    # Creating partitions of the data after shuffeling
    src = root_dir + "/" + cls # Folder to copy images from

    allFileNames = os.listdir(src)
    np.random.shuffle(allFileNames)
    train_FileNames, val_FileNames= np.split(np.array(allFileNames),
                                                              [int(len(allFileNames)* (1 - val_ratio))])


    # train_FileNames = [src+'/'+ name for name in train_FileNames.tolist()]
    val_FileNames = [src+'/' + name for name in val_FileNames.tolist()]

    print('Total images: ', len(allFileNames))
    # print('Training: ', len(train_FileNames))
    print('Validation: ', len(val_FileNames))

    # Copy-pasting images
    # for name in train_FileNames:
    #     shutil.copy(name, root_dir +'/train/' + cls)

    for name in val_FileNames:
        shutil.copy(name, root_dir +'/val/' + cls)


**Now you can run from below to test our approach**

**Make sure mounting is correct**

In [3]:
data_transforms = {
    'train': transforms.Compose([
        #transforms.RandomCrop(224),
        transforms.Resize((256,256)),
        #transforms.RandomResizedCrop(512,scale=(0.7, 1.0), ratio=(0.75, 1.3333333333333333)),
        #transforms.RandomHorizontalFlip(p=0.5),
        #transforms.RandomVerticalFlip(p=0.5),
        # transforms.RandomAffine(5, translate=(0.2,0.2), scale=(0.9,1.2), shear=(10,10), resample=False, fillcolor=0),
        # transforms.RandomPerspective(distortion_scale=0.1),
        #transforms.RandomRotation(10, resample=False, expand=False),
        #transforms.RandomErasing(p=0.5),
        #transforms.ColorJitter(brightness=0.1, contrast=0.1, saturation=0.1, hue=0.1),
        transforms.ToTensor(),
        # transforms.Normalize([0.3794, 0.3961, 0.3725], [0.1846, 0.1853, 0.1804])
    ]),
    'validation': transforms.Compose([                                    
        transforms.Resize((256, 256)),
        transforms.ToTensor(),
        # transforms.Normalize([0.3794, 0.3961, 0.3725], [0.1846, 0.1853, 0.1804])
    ]),
    'test': transforms.Compose([
        transforms.Resize((256,256)), 
        transforms.ToTensor(),
        # transforms.Normalize([0.3794, 0.3961, 0.3725], [0.1846, 0.1853, 0.1804])
    ]),
}

In [4]:
#making the dataset
train_dataset = datasets.ImageFolder("/content/drive/My Drive/GNR/dataset/train_new/train/", data_transforms['train'])
val_dataset = datasets.ImageFolder("/content/drive/My Drive/GNR/dataset/train/val/", data_transforms['validation'])


#making the dataloader
dataloaders = {}
dataloaders['train'] = torch.utils.data.DataLoader(
                                    train_dataset,                                        
                                    batch_size=100, 
                                    shuffle=True, num_workers=4) # crashes for batch size >120
dataloaders['validation'] = torch.utils.data.DataLoader(
                                    val_dataset,                                        
                                    batch_size=10,
                                    shuffle=True, num_workers=4)

dataset_sizes = {}
dataset_sizes['train'] = len(train_dataset)
dataset_sizes['validation'] = len(val_dataset)
classes = train_dataset.classes

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


In [ ]:
# print(dataset_sizes['train'])
# print(dataloaders['train'])

In [10]:

def _make_divisible(v, divisor, min_value=None):
    """
    This function is taken from the original tf repo.
    It ensures that all layers have a channel number that is divisible by 8
    It can be seen here:
    https://github.com/tensorflow/models/blob/master/research/slim/nets/mobilenet/mobilenet.py
    """
    if min_value is None:
        min_value = divisor
    new_v = max(min_value, int(v + divisor / 2) // divisor * divisor)
    # Make sure that round down does not go down by more than 10%.
    if new_v < 0.9 * v:
        new_v += divisor
    return new_v


def hard_sigmoid(x, inplace: bool = False):
    if inplace:
        return x.add_(3.).clamp_(0., 6.).div_(6.)
    else:
        return F.relu6(x + 3.) / 6.


class SqueezeExcite(nn.Module):
    def __init__(self, in_chs, se_ratio=0.25, reduced_base_chs=None,
                 act_layer=nn.ReLU, gate_fn=hard_sigmoid, divisor=4, **_):
        super(SqueezeExcite, self).__init__()
        self.gate_fn = gate_fn
        reduced_chs = _make_divisible((reduced_base_chs or in_chs) * se_ratio, divisor)
        self.avg_pool = nn.AdaptiveAvgPool2d(1)
        self.conv_reduce = nn.Conv2d(in_chs, reduced_chs, 1, bias=True)
        self.act1 = act_layer(inplace=True)
        self.conv_expand = nn.Conv2d(reduced_chs, in_chs, 1, bias=True)

    def forward(self, x):
        x_se = self.avg_pool(x)
        x_se = self.conv_reduce(x_se)
        x_se = self.act1(x_se)
        x_se = self.conv_expand(x_se)
        x = x * self.gate_fn(x_se)
        return x    

    
class ConvBnAct(nn.Module):
    def __init__(self, in_chs, out_chs, kernel_size,
                 stride=1, act_layer=nn.ReLU):
        super(ConvBnAct, self).__init__()
        self.conv = nn.Conv2d(in_chs, out_chs, kernel_size, stride, kernel_size//2, bias=False)
        self.bn1 = nn.BatchNorm2d(out_chs)
        self.act1 = act_layer(inplace=True)

    def forward(self, x):
        x = self.conv(x)
        x = self.bn1(x)
        x = self.act1(x)
        return x


class GhostModule(nn.Module):
    def __init__(self, inp, oup, kernel_size=1, ratio=2, dw_size=3, stride=1, relu=True):
        super(GhostModule, self).__init__()
        self.oup = oup
        init_channels = math.ceil(oup / ratio)
        new_channels = init_channels*(ratio-1)

        self.primary_conv = nn.Sequential(
            nn.Conv2d(inp, init_channels, kernel_size, stride, kernel_size//2, bias=False),
            nn.BatchNorm2d(init_channels),
            nn.ReLU(inplace=True) if relu else nn.Sequential(),
        )

        self.cheap_operation = nn.Sequential(
            nn.Conv2d(init_channels, new_channels, dw_size, 1, dw_size//2, groups=init_channels, bias=False),
            nn.BatchNorm2d(new_channels),
            nn.ReLU(inplace=True) if relu else nn.Sequential(),
        )

    def forward(self, x):
        x1 = self.primary_conv(x)
        x2 = self.cheap_operation(x1)
        out = torch.cat([x1,x2], dim=1)
        return out[:,:self.oup,:,:]


class GhostBottleneck(nn.Module):
    """ Ghost bottleneck w/ optional SE"""

    def __init__(self, in_chs, mid_chs, out_chs, dw_kernel_size=3,
                 stride=1, act_layer=nn.ReLU, se_ratio=0.):
        super(GhostBottleneck, self).__init__()
        has_se = se_ratio is not None and se_ratio > 0.
        self.stride = stride

        # Point-wise expansion
        self.ghost1 = GhostModule(in_chs, mid_chs, relu=True)

        # Depth-wise convolution
        if self.stride > 1:
            self.conv_dw = nn.Conv2d(mid_chs, mid_chs, dw_kernel_size, stride=stride,
                             padding=(dw_kernel_size-1)//2,
                             groups=mid_chs, bias=False)
            self.bn_dw = nn.BatchNorm2d(mid_chs)

        # Squeeze-and-excitation
        if has_se:
            self.se = SqueezeExcite(mid_chs, se_ratio=se_ratio)
        else:
            self.se = None

        # Point-wise linear projection
        self.ghost2 = GhostModule(mid_chs, out_chs, relu=False)
        
        # shortcut
        if (in_chs == out_chs and self.stride == 1):
            self.shortcut = nn.Sequential()
        else:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_chs, in_chs, dw_kernel_size, stride=stride,
                       padding=(dw_kernel_size-1)//2, groups=in_chs, bias=False),
                nn.BatchNorm2d(in_chs),
                nn.Conv2d(in_chs, out_chs, 1, stride=1, padding=0, bias=False),
                nn.BatchNorm2d(out_chs),
            )


    def forward(self, x):
        residual = x

        # 1st ghost bottleneck
        x = self.ghost1(x)

        # Depth-wise convolution
        if self.stride > 1:
            x = self.conv_dw(x)
            x = self.bn_dw(x)

        # Squeeze-and-excitation
        if self.se is not None:
            x = self.se(x)

        # 2nd ghost bottleneck
        x = self.ghost2(x)
        
        x += self.shortcut(residual)
        return x

class GhostNet(nn.Module):
    def __init__(self, cfgs, num_classes=4, width=1.0, dropout=0.2):
        super(GhostNet, self).__init__()
        # setting of inverted residual blocks
        self.cfgs = cfgs
        self.dropout = dropout

        # building first layer
        output_channel = _make_divisible(16 * width, 4)
        self.conv_stem = nn.Conv2d(3, output_channel, 3, 2, 1, bias=False)
        self.bn1 = nn.BatchNorm2d(output_channel)
        self.act1 = nn.ReLU(inplace=True)
        input_channel = output_channel

        # building inverted residual blocks
        stages = []
        block = GhostBottleneck
        for cfg in self.cfgs:
            layers = []
            for k, exp_size, c, se_ratio, s in cfg:
                output_channel = _make_divisible(c * width, 4)
                hidden_channel = _make_divisible(exp_size * width, 4)
                layers.append(block(input_channel, hidden_channel, output_channel, k, s,
                              se_ratio=se_ratio))
                input_channel = output_channel
            stages.append(nn.Sequential(*layers))

        output_channel = _make_divisible(exp_size * width, 4)
        stages.append(nn.Sequential(ConvBnAct(input_channel, output_channel, 1)))
        input_channel = output_channel
        
        self.blocks = nn.Sequential(*stages)        

        # building last several layers
        output_channel = 640
        self.global_pool = nn.AdaptiveAvgPool2d((1, 1))
        self.conv_head = nn.Conv2d(input_channel, output_channel, 1, 1, 0, bias=True)
        self.act2 = nn.ReLU(inplace=True)
        self.classifier = nn.Linear(output_channel, num_classes)
        self.act3 = nn.Softmax(dim=1)

    def forward(self, x):
        x = self.conv_stem(x)
        x = self.bn1(x)
        x = self.act1(x)
        x = self.blocks(x)
        x = self.global_pool(x)
        x = self.conv_head(x)
        x = self.act2(x)
        x = x.view(x.size(0), -1)
        if self.dropout > 0.:
            x = F.dropout(x, p=self.dropout, training=self.training)
        x = self.classifier(x)
        x = self.act3(x)
        return x
        


def ghostnet(**kwargs):
    """
    Constructs a GhostNet model
    """
    cfgs = [
        # k, t, c, SE, s 
        # stage1
        [[3,  16,  16, 0, 1]],
        # stage2
        [[3,  48,  24, 0, 2]],
        [[3,  72,  24, 0, 1]],
        # stage3
        [[5,  72,  40, 0.25, 2]],
        [[5, 120,  40, 0.25, 1]],
        # stage4
        [[3, 240,  80, 0, 2]],
        [[3, 200,  80, 0, 1],
         [3, 184,  80, 0, 1],
         [3, 184,  80, 0, 1],
         [3, 480, 112, 0.25, 1],
         [3, 672, 112, 0.25, 1]
        ],
        # stage5
        [[5, 672, 160, 0.25, 2]],
        [[5, 960, 160, 0, 1],
         [5, 960, 160, 0.25, 1],
         [5, 960, 160, 0, 1],
         [5, 960, 160, 0.25, 1]
        ]
    ]
    return GhostNet(cfgs, **kwargs)



model = ghostnet()
criterion = nn.CrossEntropyLoss() 

optimizer = torch.optim.Adam(model.parameters(), lr=0.1, weight_decay=0.1)

scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer)

In [11]:
# input = torch.rand([1,3,256,256])
# outputs = model(input)
# outputs

tensor([[0.2039, 0.2820, 0.2716, 0.2425]], grad_fn=<SoftmaxBackward0>)

In [12]:
model

GhostNet(
  (conv_stem): Conv2d(3, 16, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
  (bn1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (act1): ReLU(inplace=True)
  (blocks): Sequential(
    (0): Sequential(
      (0): GhostBottleneck(
        (ghost1): GhostModule(
          (primary_conv): Sequential(
            (0): Conv2d(16, 8, kernel_size=(1, 1), stride=(1, 1), bias=False)
            (1): BatchNorm2d(8, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
            (2): ReLU(inplace=True)
          )
          (cheap_operation): Sequential(
            (0): Conv2d(8, 8, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=8, bias=False)
            (1): BatchNorm2d(8, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
            (2): ReLU(inplace=True)
          )
        )
        (ghost2): GhostModule(
          (primary_conv): Sequential(
            (0): Conv2d(16, 8, kernel_size=(1, 1)

In [18]:

#added new parameter of start_epoch
def train_model(model, criterion, optimizer, scheduler, num_epochs,resume=None,model_name='ghosnet'):
    since = time.time()

    train_loss = []
    val_loss = []

    best_acc = 0.0
    start_epoch = 0

    for epoch in range(start_epoch, num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'validation']:#
            if phase == 'train':
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0

            ground_truth = []
            predictions = []
            confidence = []

            new_live_images = []
            new_ground_truth = []
            new_predictions = []

            
            # Iterate over data.
            temp = 0

            for inputs, labels in dataloaders[phase]:

                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                # track history if only in train
                
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)

                    _, preds = torch.max(outputs, dim=1)
                    loss = criterion(outputs, labels)

                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                    else:
                        ground_truth.append(labels)
                        predictions.append(preds)
                        # print(outputs)
                        conf_scores,idx = torch.max(outputs,dim=1)
                        # print(idx)
                        confidence.append(conf_scores)

                
                # Statistics
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)

                
            if phase == 'validation':
                scheduler.step(running_loss)
           

            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects.double() / dataset_sizes[phase]

            if phase == "train":
              train_loss.append(epoch_loss)
            elif phase == "test":
              test_loss.append(epoch_loss)
            else:
              val_loss.append(epoch_loss)

            print('{} Loss: {:.4f} Acc: {:.4f}'.format(
                phase, epoch_loss, epoch_acc))

            # deep copy the model
            if phase == 'validation' and epoch_acc > best_acc:
                best_acc = epoch_acc

                model_weight_dir = './weights_'+model_name

        print()


    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))

    return model, train_loss, val_loss

In [ ]:
model_training = train_model(model, criterion, 
                              optimizer,scheduler,
                              num_epochs=5)

In [ ]:
import matplotlib.patches as mpatches
plt.plot(model_training[1], label="Training set")
plt.plot(model_training[2], label="Validation set")
#training = mpatches.Patch(color='skyblue', label='Training')
#validation = mpatches.Patch(color='orange', label='validation')
plt.xlabel("No. of epochs")
plt.ylabel("Loss")
plt.legend()

In [ ]:
test_dataset = datasets.ImageFolder("/content/drive/MyDrive/GNR/dataset/test/", data_transforms['test'])

test_dataloader = torch.utils.data.DataLoader(
                                    test_dataset,                                        
                                    batch_size=1,
                                    shuffle=False, num_workers=4)

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


In [ ]:
correct = 0
total = 0
with torch.no_grad():
    model.eval()
    # l=[]
    # p=[]
    for data in tqdm(test_dataloader):
        images, labels = data

        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        # l.append(labels)
        # p.append(predicted)
        correct += (predicted == labels).sum().item()

print('Accuracy of the network on the 100 test images: %d %%' % (
    100 * correct / total))
# print(l)
# print(p)